<a href="https://colab.research.google.com/github/tomonari-masada/course2021-stats2/blob/main/02_PyMC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. PyMC3入門
* 今回、PyMC3を解説するに当たっては、PyMC3の公式サイトを参考にした。
 * https://docs.pymc.io/notebooks/api_quickstart.html
* というのも、参考書『Pythonで体験するベイズ推論:PyMCによるMCMC入門』のサイトにあるPyMC3の解説（[ここ](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter2_MorePyMC/Ch2_MorePyMC_PyMC3.ipynb)）が、ちょっと微妙だったためである。
 * 特にstochastic/deterministic variablesのくだり等が微妙。

## 2.0 準備

### 2.0.1 インストール
* arviz関係のエラーが出たら、ランタイムを再起動して、そこから上のセルを実行し直す。

In [1]:
!pip install arviz==0.10

     |████████████████████████████████| 1.5MB 5.8MB/s 
     |████████████████████████████████| 768kB 15.2MB/s 
     |████████████████████████████████| 4.7MB 38.3MB/s 
     |████████████████████████████████| 317kB 36.2MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [2]:
!pip install pymc3==3.8

     |████████████████████████████████| 911kB 4.2MB/s 
  Found existing installation: pymc3 3.7
    Uninstalling pymc3-3.7:
      Successfully uninstalled pymc3-3.7


### 2.0.2 復習：ベイズ的モデリングにおけるベイズ則
* $X$を観測データ、$\theta$を観測データをモデル化する確率分布のパラメータとする。
$$ p(\theta | X) = \frac{ p(X | \theta) p(\theta) }{ p(X) } $$
* ベイズ的モデリングでは、事後分布$p(\theta | X)$を知ることが目標。
 * MCMCは、事後分布から得たサンプルを通して事後分布の姿を知ろうとする時に使う。
 * 変分ベイズ推論は、事後分布を近似する代替物としての別の扱いやすい分布を通して事後分布の姿を知ろうとする時に使う。

## 2.1 PyMC3でモデルを作る

### 2.1.1 PyMC3における確率変数

* とりあえずPyMC3をインポートする。
 * エラーが出たらランタイムを再起動するなどしてみる。

In [3]:
import pymc3 as pm

* PyMC3では、いきなり確率変数を作ることはできない！
 * パラメータが1の指数分布に従う確率変数を作ろうとして、下のセルを実行すると、エラーが出るはず。

In [4]:
lambda_ = pm.Exponential("poisson_param", 1)

TypeError: ignored

* 確率変数はいきなり作るのではなく、必ずモデルという文脈の中で作る。
 * モデルはあらかじめ空のインスタンスを作っておき、「with model:」というブロックを書くか、
 * いきなり「with pm.Model() as model:」というブロックを書くかの、いずれか。

In [5]:
model = pm.Model()
with model:
    lambda_ = pm.Exponential("poisson_param", 1)

In [6]:
with pm.Model() as model:
    lambda_ = pm.Exponential("poisson_param", 1)

* 下のモデルは、まず、平均パラメータが0で標準偏差パラメータが1の正規分布に従うunobservedな確率変数muを持っている。
 * この正規分布は、事前分布。
* そして、平均パラメータが確率変数muで標準偏差パラメータが1の正規分布に従うobservedな確率変数obsを持っている。
 * この正規分布は、データを直接モデリングする確率分布。
* さらに、観測データとして、正規乱数として生成した100個の数値を指定している。
---
* このベイズ的なモデルを数式で書くと・・・
$$ \mu \sim N(0, 1) $$
$$ x \sim N(\mu, 1) $$



In [7]:
import numpy as np

model = pm.Model()
with model:
    mu = pm.Normal("mu", mu=0, sigma=1)
    obs = pm.Normal("obs", mu=mu, sigma=1, observed=np.random.randn(100))

* notebook環境では、確率変数をそのまま入力すると、それがどんな分布に従うかをきれいに表示してくれる。

In [30]:
mu

mu

In [28]:
obs

obs

* 確率変数の種類を、以下のようにして調べられる。
 
 * RVはrandom variableつまり確率変数の意。

In [8]:
model.basic_RVs

[mu, obs]

* muは値が決まっていない確率変数
 * モデルパラメータがある特定の値をとることがどのくらいありえそうかを知るために行うのがベイズ推論。

In [9]:
model.free_RVs

[mu]

* obsは値がすでに決まっている確率変数

In [10]:
model.observed_RVs

[obs]

### 2.1.2 `logp`の評価

* 観測データを100個指定しているので、muを特定の値に固定することでlogpを計算することができる。
 * logpは、後で見るように、同時確率$p(X,\theta)$の対数（データ尤度$p(X|\theta)$の対数ではない）

In [11]:
model.logp({"mu": 0})

array(-146.01981642)

* 下のように変数の文字列の名前ではなく、変数を直接指定しても、同じ意味になる。

In [12]:
model.logp({mu: 0})

array(-146.01981642)

* モデルに属するそれぞれの確率変数は、適当な初期値を設定されている。
 * この初期値は、サンプリングの出発点として用いられる。

In [13]:
mu.tag.test_value

0.0

In [14]:
obs.tag.test_value

array([-8.89038511e-02, -1.20278453e+00,  6.86192921e-02,  2.36618871e-01,
       -5.25772257e-02,  8.39865964e-01,  7.71865560e-01, -8.09038811e-01,
       -5.48341483e-01,  1.03217412e+00,  1.39740803e+00, -9.22342633e-01,
       -1.55369431e+00,  8.56322504e-01,  1.17360797e+00,  1.84145955e-02,
        9.43784946e-01, -2.69308064e+00, -2.50642106e+00,  3.08005280e-03,
        1.57817132e+00, -1.89559794e+00,  5.02758554e-01,  8.55638506e-01,
        8.69196172e-01,  1.42866920e-01, -9.05435303e-01, -1.24757259e+00,
        1.32230927e-01,  1.02373009e+00,  1.56900598e-01, -1.05605333e+00,
       -3.76366553e-01,  3.46172053e-01,  1.51975001e-01, -5.86291463e-02,
        5.71225647e-02,  7.33011134e-01, -1.17476172e+00,  2.21285046e-01,
       -2.08311613e+00, -7.20523069e-01,  1.09828816e-01, -9.36912435e-01,
       -8.71128782e-01, -2.54170982e+00, -1.46689894e+00,  5.61163340e-01,
       -9.12171164e-01,  6.77749270e-01,  4.92659758e-01,  8.81260790e-01,
        1.45948795e+00, -

* 観測されている確率変数の初期値は、設定された観測データと同じ値になっているようだ。

In [ ]:
obs.observations

array([-0.34299566, -1.57579613,  1.71517655,  0.89709066, -0.00909118,
        0.95542039,  0.91907999,  0.39416902,  0.72182413, -0.02741638,
        0.40847715,  1.52460683,  0.70718402,  0.59635839, -0.55622266,
       -0.81089366, -1.21571075, -0.82206719, -1.52018009,  0.18107061,
       -0.0284776 , -0.06128338,  0.01669006, -0.11216392,  1.45085173,
       -0.92028141, -0.19304453,  0.95774958, -1.25881075, -0.27193824,
        0.53679621,  0.100623  ,  0.26745986, -0.97452024, -0.07042789,
       -0.67537728,  1.55122915,  0.00484454, -0.994258  , -0.13928114,
       -1.42999909,  0.67644212,  0.21670315, -0.15547204, -0.78475401,
        0.53225569, -3.19543303,  1.5370369 ,  1.65988444,  0.24052496,
        0.182978  ,  1.08508894, -1.02380637,  0.21299828,  0.31325545,
        0.77292977, -0.56311335, -0.92228738,  0.52874121, -1.83968657,
        0.18263411,  0.26028187,  1.05711495, -1.35556163,  0.56606363,
       -0.19656537,  1.34774775, -1.40209482, -0.71927715,  1.31

* 下のセルの計算で、`model.logp({"mu": 0})`と同じ答えを得ることができているのは、なぜか。説明してみよう。

In [15]:
import numpy as np
from scipy.stats import norm

print(np.log(norm().pdf(obs.observations)).sum() + np.log(norm().pdf(0)))

-146.01981641534064


* 確率変数の初期値は、以下のようにして手動で設定することもできる。

In [16]:
with model:
    parameter = pm.Exponential("poisson_param", 1.0, testval=0.5)

print("parameter.tag.test_value =", parameter.tag.test_value)

parameter.tag.test_value = 0.5


* 同じ分布に従う複数の確率変数を一挙に作ることもできる。
 * 内包表記などを使って確率変数のリストを作ることは推奨されていない。
 * 下記のように、引数shapeで個数(or 形)を指定する方法が推奨されている。

In [17]:
N=10
with pm.Model() as model:
    betas = pm.Uniform("betas", 0, 1, shape=N)

In [18]:
betas.tag.test_value

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

## 2.2 PyMC3で使える確率分布

* PyMC3で使える分布は、以下の通り。
 * 単変量連続分布 https://docs.pymc.io/api/distributions/continuous.html
 * 単変量離散分布 https://docs.pymc.io/api/distributions/discrete.html
 * 多変量分布 https://docs.pymc.io/api/distributions/multivariate.html
 * 混合分布　https://docs.pymc.io/api/distributions/mixture.html

* `help`関数で各分布の説明を見ることができる。

In [19]:
help(pm.Normal)

Help on class Normal in module pymc3.distributions.continuous:

class Normal(pymc3.distributions.distribution.Continuous)
 |  Normal(name, *args, **kwargs)
 |  
 |  Univariate normal log-likelihood.
 |  
 |  The pdf of this distribution is
 |  
 |  .. math::
 |  
 |     f(x \mid \mu, \tau) =
 |         \sqrt{\frac{\tau}{2\pi}}
 |         \exp\left\{ -\frac{\tau}{2} (x-\mu)^2 \right\}
 |  
 |  Normal distribution can be parameterized either in terms of precision
 |  or standard deviation. The link between the two parametrizations is
 |  given by
 |  
 |  .. math::
 |  
 |     \tau = \dfrac{1}{\sigma^2}
 |  
 |  .. plot::
 |  
 |      import matplotlib.pyplot as plt
 |      import numpy as np
 |      import scipy.stats as st
 |      plt.style.use('seaborn-darkgrid')
 |      x = np.linspace(-5, 5, 1000)
 |      mus = [0., 0., 0., -2.]
 |      sigmas = [0.4, 1., 2., 0.4]
 |      for mu, sigma in zip(mus, sigmas):
 |          pdf = st.norm.pdf(x, mu, sigma)
 |          plt.plot(x, pdf, labe

### 2.1.3 確率変数のdeterministicなtransform
* 確率変数の値を、加減乗除や自分で定義した関数などによって変換してから、モデルの中で使うこともできる。

In [36]:
with pm.Model() as model:
    x = pm.Normal("x", mu=0, sigma=1)
    y = pm.Gamma("y", alpha=1, beta=1)
    plus_2 = x + 2
    summed = x + y
    squared = x ** 2
    sined = pm.math.sin(x)

* 変換した後の変数は`basic_RVs`には含まれない。

In [37]:
model.basic_RVs

[x, y_log__]

In [39]:
y

y

In [40]:
summed

Elemwise{add,no_inplace}.0

* 変換した後の変数がとる値もPyMC3にちゃんと追跡させるようにするには、`pm.Deterministic`を使って明示的に変換する。

In [41]:
with pm.Model() as model:
    x = pm.Normal("x", mu=0, sigma=1)
    plus_2 = x + 2
    plus_2_det = pm.Deterministic("x plus 2", x + 2)

In [42]:
model.basic_RVs

[x]

In [43]:
plus_2

Elemwise{add,no_inplace}.0

In [44]:
plus_2_det

x plus 2

## 2.3 PyMC3における推論

### 2.3.1 サンプリング

### 2.3.2 サンプリング結果の分析

### 2.3.3 変分推論